In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
from IPython.display import HTML
from base64 import b64encode
video_path = '/content/drive/MyDrive/Konkuk_CV/영상1.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

In [ ]:
class Hand_Detection():

    def __init__(self,path):
        self.path = path
        self.frames = list()
        self.detected_frames = list()
        self.cap = None

    def video_to_frames(self):
        self.cap = cv2.VideoCapture(self.path)
        while self.cap.isOpened():
            run, frame = self.cap.read()
            if not run:
                break
            img = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
            self.frames.append(img)



    def frames_rgb_to_ycr(self,frames):
        g_frames = []
        for frame in frames:
            g_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB))
        return np.array(g_frames, dtype='uint8')

    def connect_components(self,frames, original_frames, area_min):
        for k, frame in enumerate(frames):
            cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(frame)
            for i in range(1, cnt):
                (x, y, w, h, area) = stats[i]
                cent_x,cent_y = centroids[i]
                if area < area_min:
                    continue
                cv2.rectangle(original_frames[k], (int(cent_x)-w//4, int(cent_y)-h//2), (int(cent_x)+w//4, int(cent_y)+h//2), (0,0,255), 2)
        return original_frames
    '''
    def connect_components(self, frames, original_frames, area_min):
        for k, frame in enumerate(frames):
            cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(frame)

            # connected component의 통계 정보를 기반으로 area 내림차순으로 정렬
            sorted_stats = sorted(stats[1:], key=lambda x: x[4], reverse=True)

            # 가장 큰 두 개의 박스만 그리기
            for i in range(3):
                if i >= len(sorted_stats):
                    break
                (x, y, w, h, area) = sorted_stats[i]
                if area < area_min:
                    continue
                cv2.rectangle(original_frames[k], (x, y), (x+w, y+h), (0,0,255), 2)

        return original_frames
    '''
    def blur(self,frames):
        blur_frames = list()
        for k, frame in enumerate(frames):
            blur_frames.append(cv2.GaussianBlur(frame[:,:,2], (5, 5), 0))
        return np.array(blur_frames)

    def detect_foreground(self,frames):
        lower = np.array([0,140,80], dtype="uint8")
        upper = np.array([255,170,158], dtype="uint8")
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
        binary_frames = list()

        for frame in frames:
            img_hand = cv2.inRange(frame,lower,upper)
            img_hand = cv2.morphologyEx(img_hand, cv2.MORPH_CLOSE, kernel,iterations=2)
            binary_frames.append(img_hand)

        return np.array(binary_frames,dtype='uint8')

    def fit(self):
        self.video_to_frames()
        hsv_frames = self.frames_rgb_to_ycr(self.frames)
        #hsv_frames[:,:,:,2] = self.blur(hsv_frames)
        bin_frames = self.detect_foreground(hsv_frames)
        self.detected_frames = self.connect_components(bin_frames,self.frames.copy(),20000)

    def get_frames(self):
        return self.detected_frames

    def cvt_to_video(self,output_path):
        width = self.cap.get(3)
        height = self.cap.get(4)
        fps = self.cap.get(5)
        fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (int(width),int(height)))
        for frame in self.frames:
            video_writer.write(frame)

        video_writer.release()
        self.cap.release()


In [ ]:
detector = Hand_Detection('/content/drive/MyDrive/Konkuk_CV/영상1.mp4')

In [ ]:
detector.fit()

In [ ]:
detector.cvt_to_video('/content/drive/MyDrive/Konkuk_CV/output.mp4')

In [ ]:
path = '/content/drive/MyDrive/Konkuk_CV/영상1.mp4'
frames = list()
cap = cv2.VideoCapture(path)
while cap.isOpened():
    run, frame = cap.read()
    if not run:
        break
    img = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
    frames.append(img)

In [ ]:
def frames_rgb_to_hsv(frames):
    g_frames = []
    for frame in frames:
        g_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB))
    return np.array(g_frames, dtype='uint8')
ycr_frames = frames_rgb_to_hsv(frames)

In [ ]:
lower = np.array([0,140,80], dtype="uint8")
upper = np.array([255,170,158], dtype="uint8")
img_hand = cv2.inRange(ycr_frames[430],lower,upper)
cv2_imshow(img_hand)